https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/data

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import seaborn as sns
from sklearn import metrics, preprocessing
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


## Consolidated Helper Functions

In [2]:
import shutil
import requests
import base64

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

## Visualization Functions

In [3]:
    # Plot a confusion matrix.
    # cm is the confusion matrix, names are the names of the classes.
    def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks = np.arange(len(names))
        plt.xticks(tick_marks, names, rotation=45)
        plt.yticks(tick_marks, names)
        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
    

# Plot an ROC. pred - the predictions, y - the expected output.
def plot_roc(pred,y):
    fpr, tpr, _ = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

IndentationError: expected an indented block (<ipython-input-3-0c6ed2e22a06>, line 17)

## Set the desired TensorFlow output level for this example

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

## Set filepath + filename

In [ ]:
# path = "/Users/princeh/Documents/KaggleCompetition/KaggleTalkingDataFraud/mnt2/ssd/kaggle-talkingdata2/competition_files/"
# filename = os.path.join(path,"train.csv")

Read CSV

In [7]:
# chunksize = 100000
# TextFileReader = pd.read_csv(filename,na_values=['NA','?'], engine='python', low_memory = True, sep=',', chunksize=chunksize, iterator=True)
# df = pd.concat(TextFileReader, ignore_index=True)

Converting Date Cols as integers

In [8]:
# df['click_time'] =  pd.to_datetime(df.click_time).astype(np.int64)
# df['attributed_time'] =  pd.to_datetime(df.attributed_time).astype(np.int64)

# # Sort by date
# df = df.sort_values(by =['app','click_time'])

In [9]:
# df.to_hdf('talkingData.hdf','mydata',mode='w')

In [10]:
# df = pd.read_hdf('talkingData.hdf','mydata')

In [11]:
# df.dtypes

In [12]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [13]:
# df_int = df.select_dtypes(include = ['int'])
# converted_int = df_int.apply(pd.to_numeric, downcast = 'unsigned')

In [14]:
# print(mem_usage(df_int))
# print(mem_usage(converted_int))

In [15]:
# converted_int.to_hdf('converted_int.hdf','mydata',mode='w')

In [16]:
# df = pd.read_hdf('converted_int.hdf','mydata')

In [17]:
# mem_usage(df)

In [18]:
# df['pace'] = np.where((df['app'] == df['app'].shift(1)), df['click_time'] - df['click_time'].shift(1), -1)

# df['ipChanged'] = np.where(((df['ip'] == df['ip'].shift(1)) & (df['app'] == df['app'].shift(1))), 0, 1)
# df['deviceChanged'] = np.where(((df['device'] == df['device'].shift(1)) & (df['app'] == df['app'].shift(1))), 0, 1)
# df['osChanged'] = np.where(((df['os'] == df['os'].shift(1)) & (df['app'] == df['app'].shift(1))), 0, 1)
# df['channelChanged'] = np.where(((df['channel'] == df['channel'].shift(1)) & (df['app'] == df['app'].shift(1))), 0, 1)

In [19]:
# mem_usage(converted_int)

In [20]:
# df_int = df.select_dtypes(include = ['int'])
# converted_int = df_int.apply(pd.to_numeric, downcast = 'unsigned')

In [21]:
# df_float = df.select_dtypes(include=['float'])
# converted_float = df_float.apply(pd.to_numeric,downcast='float')

In [22]:
# df1 = df.drop(columns=['attributed_time','pace','ipChanged','deviceChanged','osChanged','channelChanged'])

In [23]:
# df1.to_hdf('df1.hdf','mydata',mode='w')

In [24]:
# drew = pd.merge(df1, converted_int, left_index=True, right_index=True)

In [25]:
# drew.to_hdf('drew.hdf','mydata',mode='w')

In [26]:
# draft = pd.merge(drew, converted_float, left_index=True, right_index=True)

In [27]:
# draft.drop(columns = 'pace', inplace = True)

In [28]:
# draft.to_hdf('draft.hdf','mydata',mode='w')

In [34]:
df = pd.read_hdf('draft.hdf','mydata')

In [39]:
mem_usage(df)

'6524.51 MB'

In [36]:
df = df.drop(columns = ['attributed_time'])

In [41]:
# for col in ['is_attributed','ipChanged','deviceChanged','osChanged','channelChanged']:
#     df[col] = df[col].astype('category')

In [ ]:
mem_usage(df)

In [ ]:
# df.reset_index(inplace=True)
# df.drop(columns='index',axis=0,inplace=True)
# df.index.name = 'event'

In [42]:
df['ipCount'] = 0
df['ipCount'] = df['ipCount'].apply(pd.to_numeric, downcast = 'unsigned')
df['deviceCount'] = 0
df['deviceCount'] = df['deviceCount'].apply(pd.to_numeric, downcast = 'unsigned')
df['osCount'] = 0
df['osCount'] = df['osCount'].apply(pd.to_numeric, downcast = 'unsigned')
df['channelCount'] = 0
df['channelCount'] = df['channelCount'].apply(pd.to_numeric, downcast = 'unsigned')
df['linkages'] = 0
df['linkages'] = df['linkages'].apply(pd.to_numeric, downcast = 'unsigned')

KeyboardInterrupt: 

In [ ]:
df.to_hdf('df.hdf','mydata',mode='w')

In [ ]:
df = pd.read_hdf('df.hdf','mydata')

In [ ]:
# for col in ['ipCount','deviceCount','osCount','channelCount','linkages']:
#     df[col] = df[col].apply(pd.to_numeric, downcast = 'unsigned')

In [ ]:
ipCount = []
deviceCount = []
osCount = []
channelCount = []
linkages = []

for index, row in df.iterrows():
    ipCount.append(df.loc[((df.index <= index) & (df.app == row.app)),'ip'].nunique()).apply(pd.to_numeric, downcast = 'unsigned')
    deviceCount.append(df.loc[((df.index <= index) & (df.app == row.app)),'device'].nunique()).apply(pd.to_numeric, downcast = 'unsigned')
    osCount.append(df.loc[((df.index <= index) & (df.app == row.app)),'os'].nunique()).apply(pd.to_numeric, downcast = 'unsigned')
    channelCount.append(df.loc[((df.index <= index) & (df.app == row.app)),'channel'].nunique()).apply(pd.to_numeric, downcast = 'unsigned')
    linkages.append(df.loc[((df.index <= index) & (df.ip == row.ip)),'app'].nunique()).apply(pd.to_numeric, downcast = 'unsigned')
#     print(index, row.app)

In [ ]:
df.ipCount = ipCount
df.deviceCount = deviceCount
df.osCount = osCount
df.channelCount = channelCount
df.linkages = linkages

In [ ]:
# df_ip = pd.DataFrame(df.groupby('ip')['app'].nunique())
# df_ip.columns = ['linkages']
# df_ip_links = pd.merge(df, df_ip, left_on='ip', right_index=True)
# df_consolidated = pd.merge(df_ip_links, df, left_index=True, right_index=True)

In [ ]:
df = df.drop(columns = ['ip','device','app','channel','os','click_time','attributed_time'], axis=1)

In [ ]:
# df.ip = df.ip.astype(str)
# df.app = df.app.astype(str)
# df.device = df.device.astype(str)
# df.os = df.os.astype(str)
# df.channel = df.channel.astype(str)

In [ ]:
df.head()

In [ ]:
# Encode feature vector
# df.drop(['ip', 'device', 'os', 'channel', 'click_time'],axis=1,inplace=True)
diagnosis = encode_text_index(df,'is_attributed')
num_classes = len(diagnosis)

In [ ]:
# Create the x-side (feature vectors) of the training
x, y = to_xy(df,'is_attributed')

In [ ]:
df

In [ ]:
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
# Build network
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=0,epochs=1000)
model.load_weights('best_weights.hdf5') # load weights from best model

In [ ]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_compare = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_compare, pred)
print("Final accuracy: {}".format(score))

In [ ]:
# !conda install keras

Confusion Matrix

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
cm = confusion_matrix(y_compare, pred)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm, diagnosis)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, diagnosis, title='Normalized confusion matrix')

plt.show()

In [ ]:
pred = model.predict(x_test)
pred = pred[:,1] # Only positive cases
plot_roc(pred,y_compare)